# Tutorial 20: Bayesian Deep Learning

This tutorial explores Bayesian approaches to deep learning, focusing on uncertainty quantification and probabilistic modeling.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal, Categorical
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Optional
import seaborn as sns
from scipy import stats
from sklearn.datasets import make_moons, make_circles

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Understanding Uncertainty in Deep Learning

There are two main types of uncertainty:
- **Aleatoric uncertainty**: Irreducible noise in the data
- **Epistemic uncertainty**: Uncertainty due to lack of knowledge

In [ ]:
# Generate synthetic data with different types of uncertainty
def generate_data_with_uncertainty(n_samples=200):
    """Generate data with epistemic and aleatoric uncertainty"""
    x = np.linspace(-3, 3, n_samples)
    
    # True function
    y_true = np.sin(x) + 0.3 * np.cos(2 * x)
    
    # Aleatoric uncertainty (heteroscedastic noise)
    noise_std = 0.05 + 0.15 * np.exp(-x**2)  # Higher noise at extremes
    y_aleatoric = y_true + np.random.normal(0, noise_std)
    
    # Epistemic uncertainty (remove data in certain regions)
    mask1 = (x > -1.5) & (x < -0.5)
    mask2 = (x > 0.5) & (x < 1.5)
    mask = ~(mask1 | mask2)
    x_epistemic = x[mask]
    y_epistemic = y_aleatoric[mask]
    
    return x, y_true, y_aleatoric, x_epistemic, y_epistemic, noise_std

x, y_true, y_aleatoric, x_epistemic, y_epistemic, noise_std = generate_data_with_uncertainty()

In [ ]:
# Visualize uncertainty types
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Aleatoric uncertainty
axes[0].plot(x, y_true, 'k-', label='True function', linewidth=2)
axes[0].scatter(x[::5], y_aleatoric[::5], alpha=0.6, s=30, label='Noisy observations')
axes[0].fill_between(x, y_true - 2*noise_std, y_true + 2*noise_std, 
                     alpha=0.3, label='Aleatoric uncertainty (±2σ)')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('y', fontsize=12)
axes[0].set_title('Aleatoric Uncertainty (Data Noise)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Epistemic uncertainty
axes[1].plot(x, y_true, 'k-', label='True function', linewidth=2)
axes[1].scatter(x_epistemic, y_epistemic, alpha=0.6, s=30, label='Available data')
axes[1].axvspan(-1.5, -0.5, alpha=0.2, color='red', label='High epistemic uncertainty')
axes[1].axvspan(0.5, 1.5, alpha=0.2, color='red')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('y', fontsize=12)
axes[1].set_title('Epistemic Uncertainty (Lack of Data)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Insights:")
print("• Aleatoric uncertainty: Cannot be reduced with more data")
print("• Epistemic uncertainty: Can be reduced by collecting more data")
print("• Both types are important for robust decision making")

## 2. Monte Carlo Dropout

MC Dropout is a simple yet effective method for uncertainty estimation using dropout at test time.

In [ ]:
class MCDropoutNet(nn.Module):
    """Neural network with Monte Carlo Dropout for uncertainty estimation"""
    def __init__(self, input_dim=1, hidden_dim=100, output_dim=1, dropout_rate=0.1):
        super().__init__()
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
        # Dropout layers
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x, training=True):
        # Apply dropout even during evaluation for MC Dropout
        if training:
            self.train()
        else:
            self.eval()
            # Override eval mode for dropout layers
            for m in self.modules():
                if isinstance(m, nn.Dropout):
                    m.train()
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
    def predict_with_uncertainty(self, x, n_samples=50):
        """Make predictions with uncertainty using MC Dropout"""
        self.eval()  # Set to eval mode
        predictions = []
        
        for _ in range(n_samples):
            with torch.no_grad():
                pred = self.forward(x, training=False)
                predictions.append(pred)
        
        predictions = torch.stack(predictions)
        mean = predictions.mean(dim=0)
        std = predictions.std(dim=0)
        
        return mean, std, predictions

In [ ]:
# Train MC Dropout model
def train_mc_dropout_model(x_train, y_train, epochs=1500, dropout_rate=0.2):
    """Train MC Dropout model"""
    model = MCDropoutNet(dropout_rate=dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=100, factor=0.5)
    
    x_tensor = torch.FloatTensor(x_train).unsqueeze(1).to(device)
    y_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
    
    losses = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        output = model(x_tensor)
        loss = F.mse_loss(output, y_tensor)
        
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        
        losses.append(loss.item())
        
        if epoch % 300 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
    
    return model, losses

# Train model
print("Training MC Dropout model...")
mc_model, mc_losses = train_mc_dropout_model(x_epistemic, y_epistemic)

In [ ]:
# Make predictions with uncertainty
x_test = torch.FloatTensor(x).unsqueeze(1).to(device)
mean_pred, std_pred, all_preds = mc_model.predict_with_uncertainty(x_test, n_samples=100)

# Convert to numpy
mean_pred = mean_pred.cpu().numpy().flatten()
std_pred = std_pred.cpu().numpy().flatten()
all_preds = all_preds.cpu().numpy()

# Visualize MC Dropout predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Predictions with uncertainty
ax = axes[0]
ax.plot(x, y_true, 'k-', label='True function', linewidth=2)
ax.scatter(x_epistemic, y_epistemic, alpha=0.5, s=30, label='Training data')
ax.plot(x, mean_pred, 'b-', label='MC Dropout mean', linewidth=2)
ax.fill_between(x, mean_pred - 2*std_pred, mean_pred + 2*std_pred, 
                alpha=0.3, label='Uncertainty (±2σ)')

# Highlight uncertainty regions
ax.axvspan(-1.5, -0.5, alpha=0.1, color='red')
ax.axvspan(0.5, 1.5, alpha=0.1, color='red')
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('MC Dropout Uncertainty Estimation', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Sample predictions
ax = axes[1]
ax.plot(x, y_true, 'k-', label='True function', linewidth=2)
ax.scatter(x_epistemic, y_epistemic, alpha=0.5, s=30, label='Training data')

# Plot some individual predictions
for i in range(min(20, all_preds.shape[0])):
    ax.plot(x, all_preds[i, :, 0], 'b-', alpha=0.1, linewidth=0.5)

ax.plot(x, mean_pred, 'r-', label='Mean prediction', linewidth=2)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('MC Dropout Sample Predictions', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyze uncertainty
data_regions = ~(((x > -1.5) & (x < -0.5)) | ((x > 0.5) & (x < 1.5)))
no_data_regions = ((x > -1.5) & (x < -0.5)) | ((x > 0.5) & (x < 1.5))

print(f"Average uncertainty in data regions: {std_pred[data_regions].mean():.3f}")
print(f"Average uncertainty in no-data regions: {std_pred[no_data_regions].mean():.3f}")
print(f"Uncertainty ratio: {std_pred[no_data_regions].mean() / std_pred[data_regions].mean():.2f}x")

## 3. Bayesian Neural Networks

Bayesian Neural Networks place distributions over weights instead of point estimates.

In [ ]:
class BayesianLinear(nn.Module):
    """Bayesian linear layer with weight uncertainty"""
    def __init__(self, in_features, out_features, prior_std=1.0):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Weight parameters (mean and log variance)
        self.weight_mu = nn.Parameter(torch.randn(out_features, in_features) * 0.1)
        self.weight_rho = nn.Parameter(torch.randn(out_features, in_features) * -3)
        
        # Bias parameters
        self.bias_mu = nn.Parameter(torch.zeros(out_features))
        self.bias_rho = nn.Parameter(torch.randn(out_features) * -3)
        
        # Prior distributions
        self.weight_prior = Normal(0, prior_std)
        self.bias_prior = Normal(0, prior_std)
        
        # For KL divergence
        self.kl_divergence = 0
        
    def forward(self, x):
        # Sample weights and biases using reparameterization trick
        weight_sigma = torch.log1p(torch.exp(self.weight_rho))
        weight_dist = Normal(self.weight_mu, weight_sigma)
        weight = weight_dist.rsample()
        
        bias_sigma = torch.log1p(torch.exp(self.bias_rho))
        bias_dist = Normal(self.bias_mu, bias_sigma)
        bias = bias_dist.rsample()
        
        # Compute KL divergence
        self.kl_divergence = (
            torch.distributions.kl_divergence(weight_dist, self.weight_prior).sum() +
            torch.distributions.kl_divergence(bias_dist, self.bias_prior).sum()
        )
        
        return F.linear(x, weight, bias)

class BayesianNN(nn.Module):
    """Bayesian Neural Network"""
    def __init__(self, input_dim=1, hidden_dim=50, output_dim=1, prior_std=1.0):
        super().__init__()
        self.fc1 = BayesianLinear(input_dim, hidden_dim, prior_std)
        self.fc2 = BayesianLinear(hidden_dim, hidden_dim, prior_std)
        self.fc3 = BayesianLinear(hidden_dim, output_dim, prior_std)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def kl_divergence(self):
        """Total KL divergence of the model"""
        kl = 0
        for module in self.modules():
            if isinstance(module, BayesianLinear):
                kl += module.kl_divergence
        return kl
    
    def predict_with_uncertainty(self, x, n_samples=50):
        """Make predictions with uncertainty"""
        self.eval()
        predictions = []
        
        for _ in range(n_samples):
            with torch.no_grad():
                pred = self.forward(x)
                predictions.append(pred)
        
        predictions = torch.stack(predictions)
        mean = predictions.mean(dim=0)
        std = predictions.std(dim=0)
        
        return mean, std, predictions

In [ ]:
# Train Bayesian Neural Network
def train_bayesian_nn(x_train, y_train, epochs=2000, kl_weight=0.01):
    """Train Bayesian Neural Network with variational inference"""
    model = BayesianNN(prior_std=1.0).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    x_tensor = torch.FloatTensor(x_train).unsqueeze(1).to(device)
    y_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
    
    n_data = len(x_train)
    losses = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        output = model(x_tensor)
        
        # ELBO loss = -log likelihood + KL divergence
        nll = F.mse_loss(output, y_tensor, reduction='sum')
        kl = model.kl_divergence()
        loss = nll + kl_weight * kl / n_data
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if epoch % 400 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}, "
                  f"NLL: {nll.item():.4f}, KL: {kl.item():.4f}")
    
    return model, losses

# Train Bayesian NN
print("Training Bayesian Neural Network...")
bnn_model, bnn_losses = train_bayesian_nn(x_epistemic, y_epistemic)

In [ ]:
# Make predictions
mean_bnn, std_bnn, all_preds_bnn = bnn_model.predict_with_uncertainty(x_test, n_samples=100)
mean_bnn = mean_bnn.cpu().numpy().flatten()
std_bnn = std_bnn.cpu().numpy().flatten()

# Visualize weight distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot weight distributions for first layer
ax = axes[0, 0]
weight_mu = bnn_model.fc1.weight_mu.detach().cpu().numpy().flatten()
weight_rho = bnn_model.fc1.weight_rho.detach().cpu().numpy().flatten()
weight_sigma = np.log1p(np.exp(weight_rho))

ax.hist(weight_mu, bins=30, alpha=0.5, label='Weight means', density=True)
ax.hist(weight_sigma, bins=30, alpha=0.5, label='Weight stds', density=True)
ax.set_xlabel('Value')
ax.set_ylabel('Density')
ax.set_title('BNN First Layer Weight Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# BNN predictions
ax = axes[0, 1]
ax.plot(x, y_true, 'k-', label='True function', linewidth=2)
ax.scatter(x_epistemic, y_epistemic, alpha=0.5, s=30, label='Training data')
ax.plot(x, mean_bnn, 'g-', label='BNN mean', linewidth=2)
ax.fill_between(x, mean_bnn - 2*std_bnn, mean_bnn + 2*std_bnn, 
                alpha=0.3, color='green', label='Uncertainty (±2σ)')
ax.axvspan(-1.5, -0.5, alpha=0.1, color='red')
ax.axvspan(0.5, 1.5, alpha=0.1, color='red')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Bayesian Neural Network Predictions')
ax.legend()
ax.grid(True, alpha=0.3)

# Training loss comparison
ax = axes[1, 0]
ax.plot(mc_losses, label='MC Dropout', alpha=0.7)
ax.plot(bnn_losses, label='Bayesian NN', alpha=0.7)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Loss Comparison')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

# Uncertainty comparison
ax = axes[1, 1]
ax.plot(x, std_pred, 'b-', label='MC Dropout', linewidth=2)
ax.plot(x, std_bnn, 'g-', label='Bayesian NN', linewidth=2)
ax.axvspan(-1.5, -0.5, alpha=0.1, color='red')
ax.axvspan(0.5, 1.5, alpha=0.1, color='red')
ax.set_xlabel('x')
ax.set_ylabel('Uncertainty (σ)')
ax.set_title('Uncertainty Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Deep Ensembles

Deep ensembles train multiple models independently and combine their predictions.

In [ ]:
class SimpleNN(nn.Module):
    """Simple neural network for ensemble"""
    def __init__(self, input_dim=1, hidden_dim=50, output_dim=1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
        # Different initialization for diversity
        self.reset_parameters()
        
    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class DeepEnsemble:
    """Deep ensemble for uncertainty estimation"""
    def __init__(self, n_models=5, **model_kwargs):
        self.n_models = n_models
        self.models = [SimpleNN(**model_kwargs).to(device) for _ in range(n_models)]
        
    def train(self, x_train, y_train, epochs=1500):
        """Train ensemble members independently"""
        x_tensor = torch.FloatTensor(x_train).unsqueeze(1).to(device)
        y_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
        
        all_losses = []
        
        for i, model in enumerate(self.models):
            print(f"\nTraining model {i+1}/{self.n_models}")
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=100)
            
            losses = []
            
            for epoch in range(epochs):
                model.train()
                optimizer.zero_grad()
                
                # Add different random seeds for diversity
                torch.manual_seed(42 + i * 1000 + epoch)
                
                output = model(x_tensor)
                loss = F.mse_loss(output, y_tensor)
                
                loss.backward()
                optimizer.step()
                scheduler.step(loss)
                
                losses.append(loss.item())
                
                if epoch % 500 == 0:
                    print(f"  Epoch {epoch}, Loss: {loss.item():.4f}")
            
            all_losses.append(losses)
        
        return all_losses
    
    def predict_with_uncertainty(self, x):
        """Make predictions with uncertainty using ensemble"""
        predictions = []
        
        for model in self.models:
            model.eval()
            with torch.no_grad():
                pred = model(x)
                predictions.append(pred)
        
        predictions = torch.stack(predictions)
        mean = predictions.mean(dim=0)
        
        # Total uncertainty from ensemble variance
        epistemic_std = predictions.std(dim=0)
        
        return mean, epistemic_std, predictions

In [ ]:
# Train deep ensemble
print("Training Deep Ensemble...")
ensemble = DeepEnsemble(n_models=5)
ensemble_losses = ensemble.train(x_epistemic, y_epistemic)

# Make predictions
mean_ensemble, std_ensemble, ensemble_preds = ensemble.predict_with_uncertainty(x_test)
mean_ensemble = mean_ensemble.cpu().numpy().flatten()
std_ensemble = std_ensemble.cpu().numpy().flatten()
ensemble_preds = ensemble_preds.cpu().numpy()

## 5. Comparison of Methods

In [ ]:
# Comprehensive comparison visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# All methods comparison
ax = axes[0, 0]
ax.plot(x, y_true, 'k-', label='True function', linewidth=3)
ax.scatter(x_epistemic[::3], y_epistemic[::3], alpha=0.3, s=20, color='gray', label='Training data')

methods = [
    ('MC Dropout', mean_pred, std_pred, 'blue'),
    ('Bayesian NN', mean_bnn, std_bnn, 'green'),
    ('Deep Ensemble', mean_ensemble, std_ensemble, 'red')
]

for name, mean, std, color in methods:
    ax.plot(x, mean, color=color, label=f'{name}', linewidth=2, alpha=0.8)

ax.axvspan(-1.5, -0.5, alpha=0.1, color='red')
ax.axvspan(0.5, 1.5, alpha=0.1, color='red')
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Method Comparison - Predictions', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Uncertainty comparison
ax = axes[0, 1]
for name, _, std, color in methods:
    ax.plot(x, std, color=color, label=name, linewidth=2)

ax.axvspan(-1.5, -0.5, alpha=0.1, color='red')
ax.axvspan(0.5, 1.5, alpha=0.1, color='red')
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('Uncertainty (σ)', fontsize=12)
ax.set_title('Uncertainty Estimates', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Bar plot of uncertainty ratios
ax = axes[0, 2]
uncertainty_ratios = []
method_names = []

for name, _, std, _ in methods:
    ratio = std[no_data_regions].mean() / std[data_regions].mean()
    uncertainty_ratios.append(ratio)
    method_names.append(name)

bars = ax.bar(method_names, uncertainty_ratios, color=['blue', 'green', 'red'], alpha=0.7)
ax.axhline(y=1, color='black', linestyle='--', alpha=0.5)
ax.set_ylabel('Uncertainty Ratio\n(No Data / Data)', fontsize=12)
ax.set_title('Epistemic Uncertainty Detection', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, ratio in zip(bars, uncertainty_ratios):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
            f'{ratio:.2f}', ha='center', va='bottom')

# Individual prediction samples
for idx, (name, mean, std, color, preds) in enumerate([
    ('MC Dropout', mean_pred, std_pred, 'blue', all_preds),
    ('Bayesian NN', mean_bnn, std_bnn, 'green', None),
    ('Deep Ensemble', mean_ensemble, std_ensemble, 'red', ensemble_preds)
]):
    ax = axes[1, idx]
    ax.plot(x, y_true, 'k-', linewidth=2, alpha=0.5)
    ax.scatter(x_epistemic[::3], y_epistemic[::3], alpha=0.3, s=20, color='gray')
    
    if preds is not None and name != 'Bayesian NN':
        # Plot individual predictions
        n_samples = min(20, preds.shape[0])
        for i in range(n_samples):
            if name == 'MC Dropout':
                ax.plot(x, preds[i, :, 0], color=color, alpha=0.1, linewidth=0.5)
            else:  # Deep Ensemble
                ax.plot(x, preds[i, :, 0], color=color, alpha=0.3, linewidth=1)
    
    ax.plot(x, mean, color=color, linewidth=2, label='Mean')
    ax.fill_between(x, mean - 2*std, mean + 2*std, color=color, alpha=0.2)
    
    ax.axvspan(-1.5, -0.5, alpha=0.1, color='red')
    ax.axvspan(0.5, 1.5, alpha=0.1, color='red')
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title(f'{name} - Individual Predictions', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

## 6. Out-of-Distribution Detection

A key application of uncertainty estimation is detecting out-of-distribution (OOD) data.

In [ ]:
# Generate out-of-distribution data
x_ood = np.linspace(4, 6, 50)
x_ood_tensor = torch.FloatTensor(x_ood).unsqueeze(1).to(device)

# Get predictions for OOD data
ood_results = {}

# MC Dropout
mean_mc_ood, std_mc_ood, _ = mc_model.predict_with_uncertainty(x_ood_tensor)
ood_results['MC Dropout'] = (mean_mc_ood.cpu().numpy().flatten(), 
                             std_mc_ood.cpu().numpy().flatten())

# Bayesian NN
mean_bnn_ood, std_bnn_ood, _ = bnn_model.predict_with_uncertainty(x_ood_tensor)
ood_results['Bayesian NN'] = (mean_bnn_ood.cpu().numpy().flatten(),
                              std_bnn_ood.cpu().numpy().flatten())

# Deep Ensemble
mean_ens_ood, std_ens_ood, _ = ensemble.predict_with_uncertainty(x_ood_tensor)
ood_results['Deep Ensemble'] = (mean_ens_ood.cpu().numpy().flatten(),
                                std_ens_ood.cpu().numpy().flatten())

In [ ]:
# Visualize OOD detection
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Predictions including OOD region
ax = axes[0]
x_full = np.concatenate([x, x_ood])

# Training data region
ax.axvspan(x_epistemic.min(), x_epistemic.max(), alpha=0.1, color='green', 
           label='Training data region')
ax.axvspan(x_ood.min(), x_ood.max(), alpha=0.1, color='red',
           label='OOD region')

for (name, mean, std, color), (_, (mean_ood, std_ood)) in zip(methods, ood_results.items()):
    mean_full = np.concatenate([mean, mean_ood])
    ax.plot(x_full, mean_full, color=color, label=name, linewidth=2)

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Predictions with OOD Region', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Uncertainty for OOD detection
ax = axes[1]
ax.axvspan(x_epistemic.min(), x_epistemic.max(), alpha=0.1, color='green')
ax.axvspan(x_ood.min(), x_ood.max(), alpha=0.1, color='red')

for (name, _, std, color), (_, (_, std_ood)) in zip(methods, ood_results.items()):
    std_full = np.concatenate([std, std_ood])
    ax.plot(x_full, std_full, color=color, label=name, linewidth=2)

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('Uncertainty (σ)', fontsize=12)
ax.set_title('Uncertainty-based OOD Detection', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate OOD detection metrics
print("OOD Detection Performance:")
print("-" * 50)

for name, (_, std), (_, (_, std_ood)) in zip([m[0] for m in methods], 
                                              [m[1:3] for m in methods],
                                              ood_results.items()):
    # Use 95th percentile of in-distribution uncertainty as threshold
    threshold = np.percentile(std, 95)
    ood_detected = np.mean(std_ood > threshold)
    
    # Average uncertainty increase
    unc_increase = std_ood.mean() / std.mean()
    
    print(f"{name:15} - OOD detection rate: {ood_detected:6.2%}, "
          f"Uncertainty increase: {unc_increase:.2f}x")

## 7. Uncertainty Calibration

Well-calibrated uncertainties mean that the predicted confidence matches the actual accuracy.

In [ ]:
# Evaluate calibration
def plot_calibration(predictions, uncertainties, true_values, name, ax):
    """Plot calibration of uncertainty estimates"""
    # Calculate errors
    errors = np.abs(predictions - true_values)
    
    # Create calibration plot
    ax.scatter(uncertainties, errors, alpha=0.5, s=20)
    
    # Perfect calibration lines
    max_unc = uncertainties.max()
    ax.plot([0, max_unc], [0, max_unc], 'r--', label='σ calibration', linewidth=2)
    ax.plot([0, max_unc], [0, 2*max_unc], 'r:', label='2σ calibration', linewidth=1)
    
    # Calculate coverage
    within_1sigma = np.mean(errors <= uncertainties)
    within_2sigma = np.mean(errors <= 2*uncertainties)
    
    ax.set_xlabel('Predicted Uncertainty (σ)')
    ax.set_ylabel('Actual Error')
    ax.set_title(f'{name}\n1σ: {within_1sigma:.1%}, 2σ: {within_2sigma:.1%}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    return within_1sigma, within_2sigma

# Create calibration plots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

calibration_results = []
y_test_true = np.sin(x) + 0.3 * np.cos(2 * x)

for idx, (name, mean, std, _) in enumerate(methods):
    coverage_1s, coverage_2s = plot_calibration(mean, std, y_test_true, name, axes[idx])
    calibration_results.append((name, coverage_1s, coverage_2s))

plt.tight_layout()
plt.show()

# Print calibration summary
print("Calibration Summary:")
print("-" * 60)
print(f"{'Method':15} {'1σ Coverage':>15} {'2σ Coverage':>15} {'Status':>15}")
print("-" * 60)

for name, cov_1s, cov_2s in calibration_results:
    # Expected: ~68.3% for 1σ, ~95.4% for 2σ
    status = "Well-calibrated" if abs(cov_1s - 0.683) < 0.1 else "Miscalibrated"
    print(f"{name:15} {cov_1s:14.1%} {cov_2s:14.1%} {status:>15}")

print("\nExpected:        68.3%          95.4%")

## 8. Summary and Best Practices

In [ ]:
# Create summary comparison table
fig, ax = plt.subplots(figsize=(12, 6))
ax.axis('tight')
ax.axis('off')

# Method comparison data
comparison_data = [
    ['Method', 'Computational Cost', 'Implementation', 'Uncertainty Quality', 'Best Use Case'],
    ['MC Dropout', 'Low', 'Easy', 'Good', 'Quick uncertainty estimates'],
    ['Bayesian NN', 'Medium', 'Complex', 'Excellent', 'When accuracy is critical'],
    ['Deep Ensemble', 'High', 'Simple', 'Excellent', 'Best empirical performance'],
]

# Create table
table = ax.table(cellText=comparison_data[1:], colLabels=comparison_data[0],
                cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 2)

# Style the table
for i in range(len(comparison_data[0])):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Color code rows
colors = ['#E3F2FD', '#F3E5F5', '#FFEBEE']
for i in range(1, len(comparison_data)):
    for j in range(len(comparison_data[0])):
        table[(i, j)].set_facecolor(colors[i-1])

ax.set_title('Bayesian Deep Learning Methods Comparison', fontsize=16, pad=20)
plt.show()

print("\nKey Takeaways:")
print("=" * 50)
print("1. Uncertainty quantification is crucial for reliable AI systems")
print("2. Different methods have different trade-offs")
print("3. MC Dropout: Easy to implement, good for most applications")
print("4. Bayesian NN: Principled approach, best theoretical foundation")
print("5. Deep Ensembles: Often best empirical performance")
print("6. Always validate uncertainty estimates on your specific problem")
print("7. Consider computational constraints when choosing a method")
print("\nApplications: Medical AI, autonomous driving, financial modeling,")
print("              scientific discovery, and any safety-critical system.")